In [ ]:
So, we can mitigate the error of the circuit representing our trotterized simulation of a three-particle Heisenberg Hamiltonian.


But in the IBM example, we do not run the simulation circuit directly. But we derive state_tomography circuits from it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit
from mitiq.interface.mitiq_qiskit import qiskit_utils
from mitiq import Observable, PauliString
from typing import Any, Callable, Optional, Sequence, Union
import numpy as np
from scipy.optimize import curve_fit

from mitiq import Executor, Observable, QPROGRAM, QuantumResult
from mitiq.cdr import (
    generate_training_circuits,
    linear_fit_function,
    linear_fit_function_no_intercept,
    is_clifford,
)
from mitiq.zne.scaling import fold_gates_at_random
from mitiq import Observable, PauliString

# 2. Define an executor
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import QuantumCircuit, execute, Aer
import qiskit.providers.aer.noise as noise

# Error probabilities
prob_1 = 0.005  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

def sim_noise(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obs.matrix(), noise_model, 4096)

def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)


def add_XX(qc, t, i):
    qc.ry(np.pi/2,[0+2*i,1+2*i])
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    qc.ry(-np.pi/2,[0+2*i,1+2*i])
    
def add_YY(qc, t, i):
    qc.rx(np.pi/2,[0+2*i,1+2*i])
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    qc.rx(-np.pi/2,[0+2*i,1+2*i])
    
def add_ZZ(qc, t, i):
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    

def add_trotter_step(qc, t):
    
    # Combine subcircuits into a single multiqubit gate representing a single trotter step
    num_qubits = 3

    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

    for i in range(0, num_qubits - 1):
        add_ZZ(qc, t, 1+i)
        add_YY(qc, t, 1+i)
        add_XX(qc, t, 1+i)
        
def get_circuit(steps=4):
    
    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    trotter_steps = steps  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    #qr = QuantumRegister(7)
    qc = QuantumCircuit(7)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        add_trotter_step(qc, target_time/trotter_steps)
        #qc.append(Trot_gate, [qr[1], qr[3], qr[5]])

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    #qc = qc.bind_parameters({t: })

    # Generate state tomography circuits to evaluate fidelity of simulation
    st_qcs = state_tomography_circuits(qc, [1,3,5])

    return qc
    #return st_qcs[0]


#get_circuit().draw()  # only view trotter gates

/tmp/ipykernel_137/2527742239.py:13: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter


In [5]:

qc = get_circuit()

# Generate state tomography circuits to evaluate fidelity of simulation
st_qcs = state_tomography_circuits(qc, [1,3,5])

# Display circuit for confirmation
# st_qcs[-1].decompose().draw()  # view decomposition of trotter gates
len(st_qcs)

27

In [ ]:
These are 27 circuits. So, let's see what we do with these circuits,

In [8]:
# load IBMQ Account data

IBMQ.save_account("TOKEN")  # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
provider = IBMQ.load_account()
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')
# properties = jakarta.properties()

# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

# Noiseless simulated backend
sim = QasmSimulator()

configrc.store_credentials:WARNING:2022-03-02 13:47:04,973: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-03-02 13:47:05,454: Credentials are already in use. The existing account in the session will be replaced.


In [9]:
shots = 8192
reps = 8
backend = sim_noisy_jakarta

jobs = []
for _ in range(reps):
    # execute
    job = execute(st_qcs, backend, shots=shots)
    print('Job ID', job.job_id())
    jobs.append(job)

# ...
    

Job ID 27b8bf6f-c0f3-4838-8a18-1b5752e42609


We run all of them  8192 times in eight repitions. So, we end up with 8 jobs to run each having 27*8192 circuit executions. 

That's not the problem. We have a computer doing this for us.

But let's see what we do with the results.

For each job, we get as the result an array of dictionaries. Each dictionary contains the counts of one of the 27 circuits.

“This performs measurement in the Pauli-basis resulting in :math: 3^n circuits for an n-qubit state tomography experiment.”
$3^n$ is a lower bound on the number of operators which have to be measured in order to perform the state tomography? (X, Y, Z for each qubit.)
So, for three qubits, we have run each 

In [12]:

# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    print (result.get_counts(), len(result.get_counts()))
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

# Compute tomography fidelities for each repetition
fids = []
for job in jobs:
    fid = state_tomo(job.result(), st_qcs)
    fids.append(fid)
    
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids)))

[{'101': 937, '001': 1080, '010': 1020, '000': 1015, '011': 1132, '111': 1023, '110': 1022, '100': 963}, {'001': 976, '010': 1040, '110': 1050, '100': 956, '101': 1012, '000': 1017, '111': 1049, '011': 1092}, {'010': 355, '001': 313, '101': 1596, '100': 1709, '110': 1741, '000': 358, '111': 1779, '011': 341}, {'110': 1026, '100': 1029, '101': 1036, '001': 1032, '010': 1048, '011': 1007, '000': 979, '111': 1035}, {'010': 1054, '001': 1028, '101': 983, '110': 989, '100': 984, '000': 968, '111': 1111, '011': 1075}, {'001': 361, '010': 354, '100': 1715, '110': 1707, '101': 1656, '000': 331, '111': 1729, '011': 339}, {'111': 1716, '000': 360, '011': 1700, '010': 1667, '001': 362, '101': 368, '110': 1630, '100': 389}, {'100': 408, '110': 1691, '010': 1662, '101': 371, '001': 356, '000': 370, '011': 1689, '111': 1645}, {'001': 132, '010': 538, '000': 121, '111': 2802, '011': 553, '101': 602, '110': 2777, '100': 667}, {'110': 1082, '100': 969, '001': 987, '101': 1002, '010': 1077, '111': 1051,

we put them into a StateTomographyFitter and use the least squares method to finally calculate the state_fidelity. Finally, the overall fidelity is the means of the fidelity of each repition. So, for simplicity, we now consider only a single repitition.

The question is how does the StateTomographyFitter work and how can we mitigate the measurements?

Quantum tomography or quantum state tomography is the process by which a quantum state is reconstructed using measurements on an ensemble of identical quantum states.[1] The source of these states may be any device or system which prepares quantum states either consistently into quantum pure states or otherwise into general mixed states. To be able to uniquely identify the state, the measurements must be tomographically complete. That is, the measured operators must form an operator basis on the Hilbert space of the system, providing all the information about the state. Such a set of observations is sometimes called a quorum.

state tomography aims to extract all possible information about the state that are contained in the density operator.  the structure of the density matrix and showed that knowledge of this operator suffices to describe a system in total. Behaviour under certain operations and measurement outcomes can be calculated. This means that from the measurement of selected properties of a system, other can be inferred and this is what tomography is about.

So, we look at the quantum sytem from  each possible viewpoint angle.

The StateTomographyFitter is part of the [qiskit-ignis](https://github.com/Qiskit/qiskit-ignis/blob/master/qiskit/ignis/verification/tomography/fitters/state_fitter.py). It calls the fit function from its parent class [base_fitter](https://github.com/Qiskit/qiskit-ignis/blob/master/qiskit/ignis/verification/tomography/fitters/base_fitter.py). Without going into too much detail, [the base fitter](https://github.com/Qiskit/qiskit-ignis/blob/master/qiskit/ignis/verification/tomography/fitters/base_fitter.py#L271) collects the counts of the results.

We don't look too much into it  because I don't think that IBM allows us to mess around with it. So, instead, we need to reflect our error mitigation in the counts.

But this could be a problem because the CDR does not work on the measurement level but mitigates the expectation value of an observable. So, what we could do is to take each of the tomography circuits and apply the CDR on it. But this wouldn't help us.



![](./assets/cdr_draw.png)

In [ ]:
Because we need the mitigated measurements, not the expectation value. 
So, what we would need to do is a linear regression of matrices.



In [ ]:
Measurement Error Mitigation: https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html


https://machinelearningmastery.com/curve-fitting-with-python/